# Simulating Circuit

In this section, we will explain how to simulate quantum circuits in MQC3.
Furthermore, we will also introduce how to visualize the quantum states in the form of Wigner functions.

## Simulating circuit representation

First, we create the teleportation circuit below.

![teleportation](_images/circuit_repr_teleportation.svg)

In [ ]:
# pyright: reportUnusedExpression=false

from math import pi

import numpy as np

from mqc3.circuit import CircuitRepr
from mqc3.circuit.ops.intrinsic import Displacement, Measurement
from mqc3.circuit.ops.std import BeamSplitter
from mqc3.circuit.state import BosonicState, GaussianState
from mqc3.feedforward import feedforward


# Define feedforward functions.
@feedforward
def displace_x(x):
    from math import sqrt  # noqa:PLC0415

    return sqrt(2) * x


@feedforward
def displace_p(p):
    from math import sqrt  # noqa:PLC0415

    return -sqrt(2) * p


# Construct the teleportation circuit.
circuit = CircuitRepr("teleportation")
circuit.Q(1, 2) | BeamSplitter(theta=pi / 4, phi=pi)
circuit.Q(0, 1) | BeamSplitter(theta=pi / 4, phi=pi)
# Measure modes 0 and 1.
m0 = circuit.Q(0) | Measurement(theta=pi / 2)
m1 = circuit.Q(1) | Measurement(theta=0)
# Apply displacement with feedforward.
circuit.Q(2) | Displacement(displace_x(m0), displace_p(m1))
# Set initial states.
coeff = np.array([1.0 + 0.0j])
mode0_x = 3.0
mode0_p = -1.5
mean = np.array([mode0_x + 0.0j, mode0_p + 0.0j])
# Set the initial state of the teleported mode.
circuit.set_initial_state(0, BosonicState(coeff, [GaussianState(mean, GaussianState.vacuum().cov)]))
# Set the initial states of modes 1 and 2.
circuit.set_initial_state(1, BosonicState(coeff, [GaussianState.squeezed(r=1.5, phi=pi / 2)]))  # p-squeezed.
circuit.set_initial_state(2, BosonicState(coeff, [GaussianState.squeezed(r=1.5, phi=0)]))  # x-squeezed.

circuit

### Visualizing the initial state modes

A mode in a {py:class}`~mqc3.circuit.state.BosonicState` object can be visualized as a wigner function, using {py:mod}`~mqc3.circuit.visualize_state` module.
This module provides the following functions.

* {py:func}`~mqc3.circuit.visualize_state.meshed_wigner`: Calculate the Wigner function of a specified mode, which is discretized according to the specified mesh for quadrature variables x and p.
* {py:func}`~mqc3.circuit.visualize_state.plot_wigner`: Plot the discretized Wigner function of a specified mode calculated by {py:func}`~mqc3.circuit.visualize_state.meshed_wigner` internally.

In this section, the usage of {py:func}`~mqc3.circuit.visualize_state.plot_wigner` function is demonstrated.
Now let's visualize the Wigner function of each mode in the circuit.

In [ ]:
from numpy import linspace

from mqc3.circuit.visualize_state import make_wigner_figure

input_mode = circuit.get_initial_state(0)
assert isinstance(input_mode, BosonicState)
make_wigner_figure(state=input_mode, xvec=linspace(-10, 10, 401), pvec=linspace(-10, 10, 401));

In [ ]:
p_squeezed_mode = circuit.get_initial_state(1)
assert isinstance(p_squeezed_mode, BosonicState)
make_wigner_figure(state=p_squeezed_mode, xvec=linspace(-10, 10, 401), pvec=linspace(-10, 10, 401));

In [ ]:
x_squeezed_mode = circuit.get_initial_state(2)
assert isinstance(x_squeezed_mode, BosonicState)
make_wigner_figure(state=x_squeezed_mode, xvec=linspace(-10, 10, 401), pvec=linspace(-10, 10, 401));

## Configuring client

To simulate a circuit using the MQC3 SDK, you must first create an instance of {py:class}`~.mqc3.client.SimulatorClient`.
This client allows you to specify the connection endpoint, API token, simulation mode (remote or local), and other simulation parameters.

````{note}
In remote simulation mode, {py:class}`~.mqc3.client.SimulatorClient` can simulate all representations.
In local simulation mode, {py:class}`~.mqc3.client.SimulatorClient` can simulate only circuit representations.
````

````{note}
If simulating graph representations or machinery representation, you can configure the squeezing level of resources by setting {py:attr}`~mqc3.client.SimulatorClient.resource_squeezing_level`.
The default value is 10.0. A higher squeezing level results in reduced noise, which is caused by the imperfection of the EPR state. See [theory](theory.md) for more details.
````


Let’s create a {py:class}`~.mqc3.client.SimulatorClient` instance as shown below.

In [ ]:
from mqc3.client import SimulatorClient, SimulatorClientResult

client = SimulatorClient()

If you simulate the circuit on a remote server, set the `remote` attribute to `True` and provide your API token via the `token` attribute.
On the other hand, if you want to simulate locally, set the `remote` attribute to `False`.
In this case, you do not need to set the `token`.

````{note}
To run simulations locally, you must have [Strawberry Fields](https://strawberryfields.ai) installed.
````


In [ ]:
# client.remote = True
# client.token = "YOUR_API_TOKEN"

client.remote = False

You can specify how many times the circuit will be executed by setting the `n_shots` attribute.

In [ ]:
# Run the circuit for hundred times
client.n_shots = 100

To control whether to save the quantum states after simulation, use the `state_save_policy` attribute.  
It accepts one of the following values: `"none"`, `"first_only"`, or `"all"`.

- `"none"`: Do not save any states.
- `"first_only"`: Save only the state from the first shot only.
- `"all"`: Save the states from all shots.

````{note}
If the circuit does not contain any measurement operations, the simulation is **deterministic**.
In such cases, it is recommended to set `shots=1` and `state_save_policy="all"`.
````

In [ ]:
client.state_save_policy = "all"

## Simulating circuit

You can simulate the circuit with {py:func}`~mqc3.execute.execute`.
This function takes a circuit and a client as arguments and returns the execution result.

In [ ]:
from mqc3.execute import execute

result = execute(circuit, client)

In [ ]:
type(result.client_result)

{py:class}`~mqc3.client.SimulatorClientResult` has the following attributes.

| Attribute | Type | Description |
| --------- | ---- | ----------- |
| {py:attr}`~mqc3.client.SimulatorClientResult.execution_details` | {py:class}`~mqc3.client.simulator_client.ExecutionDetails` | Versions and timestamps related to a simulation. |
| {py:attr}`~mqc3.client.SimulatorClientResult.circuit_result`   | {py:class}`~mqc3.circuit.CircuitResult` | Measurement results after circuit simulation. |
| {py:attr}`~mqc3.client.SimulatorClientResult.graph_result`   | {py:class}`~mqc3.graph.GraphResult` | Measurement results after graph simulation. |
| {py:attr}`~mqc3.client.SimulatorClientResult.machinery_result`   | {py:class}`~mqc3.machinery.MachineryResult` | Measurement results after machinery simulation. |
| {py:attr}`~mqc3.client.SimulatorClientResult.states`           | list of {py:class}`~mqc3.circuit.state.BosonicState` | The quantum states after circuit simulation. If simulating a graph representation or a machinery representation, this attribute is empty. If `state_save_policy` is `"none"`, this list is empty. If `state_save_policy` is `"first_only"`, the list contains only one element. If `state_save_policy` is `"all"`, the list contains one state per shot. |
| {py:attr}`~mqc3.client.SimulatorClientResult.execution_result` | {py:class}`~mqc3.circuit.CircuitResult`, {py:class}`~mqc3.graph.GraphResult` or {py:class}`~mqc3.machinery.MachineryResult` | Execution result. |
| {py:attr}`~mqc3.client.SimulatorClientResult.wait_time` | `timedelta` or None | The waiting time until the job starts to run. |
| {py:attr}`~mqc3.client.SimulatorClientResult.execution_time`  | `timedelta` or None | The time to execute a representation. |
| {py:attr}`~mqc3.client.SimulatorClientResult.total_time`  | `timedelta` or None | The total time to execute a representation. |
| {py:attr}`~mqc3.client.SimulatorClientResult.n_shots`   | `int` | The number of shots. |

When you simulate the circuit representation, you can get the measurement result of the circuit representation for any shot by using index access.

In [ ]:
from pprint import pprint

pprint(result[0])  # Get the 0-th shot result

In [ ]:
pprint(result[1])  # Get the 1-st shot result

You can access the quantum states after circuit simulation via {py:attr}`~mqc3.client.SimulatorClientResult.states`.

In this example, the executed circuit is a teleportation circuit that transfers the state of mode 0 to mode 2.  
We can verify whether the `mean` of mode 2 in the final state matches the initial `mean` of mode 0.

In [ ]:
error_threshold = 1.0

assert isinstance(result.client_result, SimulatorClientResult)
for state in result.client_result.states:
    mean = state.get_gaussian_state(0).mean
    cov = state.get_gaussian_state(0).cov
    teleported_x = mean[2].real
    teleported_p = mean[5].real

    assert abs(teleported_x - mode0_x) < error_threshold
    assert abs(teleported_p - mode0_p) < error_threshold

print("Teleportation successful!")

### Visualizing the output mode

To see the effect of the teleportation protocol, we visualize the Wigner function of the output mode in the first shot.

In [ ]:
client_result = result.client_result
assert isinstance(client_result, SimulatorClientResult)
final_state = client_result.states[0]
output_mode = final_state.extract_mode(2)
make_wigner_figure(state=output_mode, xvec=linspace(-10, 10, 401), pvec=linspace(-10, 10, 401));

Since the initial modes 1 and 2 have finite squeezing levels, output mode 2 differs slightly and randomly from input mode 0.